*FLAML test*
Testa FLAML på travet 

In [10]:
import pandas as pd 
import numpy as np 
from IPython.display import display 

df = pd.read_csv('..\\all_data.csv')   

In [11]:
# prepare
# Set a smooth mean value to  features in df
def calc_smooth_mean(X, by, y, tot_mean,m=100,):
    # print(by, y, m)
    # Compute the number of values and the mean of each group
    agg = X.groupby(by)[y].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + (m * tot_mean)) / (counts + m)
    # print('used m = ',m)
    return smooth.to_dict()

def transf_kusk_häst(df,pref='',m=300,the_map=None):
    
    df[pref+'ekipage'] = df[pref+ 'kusk'].str.cat(df['häst'], sep =", ")  # concatenate 'häst' and 'kusk' into one column
    # print('skapade ekipage', df[pref+'ekipage'].shape)
    
    if not the_map:
        the_map = calc_smooth_mean(df, by=pref+'ekipage', y='plac',m=50,tot_mean=df.plac.mean()) # make numeric with Target encoding with smooth mean
        
    df['ekipage'].map(the_map) 
    # print('ekipage klart')
        
    df.drop([pref+'kusk'],axis=1,inplace=True)
    return the_map, df

the_map,df = transf_kusk_häst(df,pref='', m=300)
_,df = transf_kusk_häst(df,pref='h1_', m=300, the_map=the_map)
_,df = transf_kusk_häst(df,pref='h2_', m=300, the_map=the_map)
_,df = transf_kusk_häst(df,pref='h3_', m=300, the_map=the_map)
_,df = transf_kusk_häst(df,pref='h4_', m=300, the_map=the_map)
_,df = transf_kusk_häst(df,pref='h5_', m=300, the_map=the_map)
df.drop(['häst'],axis=1,inplace=True)
df.columns

Index(['datum', 'avd', 'bana', 'streck', 'vodds', 'podds', 'kr', 'spår',
       'dist', 'lopp_dist', 'start', 'ålder', 'kön', 'plac', 'pris', 'h1_dat',
       'h1_bana', 'h1_spår', 'h1_plac', 'h1_pris', 'h1_odds', 'h1_kmtid',
       'h2_dat', 'h2_bana', 'h2_spår', 'h2_plac', 'h2_pris', 'h2_odds',
       'h2_kmtid', 'h3_dat', 'h3_bana', 'h3_spår', 'h3_plac', 'h3_pris',
       'h3_odds', 'h3_kmtid', 'h4_dat', 'h4_bana', 'h4_spår', 'h4_plac',
       'h4_pris', 'h4_odds', 'h4_kmtid', 'h5_dat', 'h5_bana', 'h5_spår',
       'h5_plac', 'h5_pris', 'h5_odds', 'h5_kmtid', 'h1_dist', 'h2_dist',
       'h3_dist', 'h4_dist', 'h5_dist', 'bins', 'h1_auto', 'h2_auto',
       'h3_auto', 'h4_auto', 'h5_auto', 'h1_perf', 'h2_perf', 'h3_perf',
       'h4_perf', 'h5_perf', 'senast', 'delta1', 'delta2', 'delta3', 'delta4',
       'startnr', 'ekipage', 'h1_ekipage', 'h2_ekipage', 'h3_ekipage',
       'h4_ekipage', 'h5_ekipage'],
      dtype='object')

In [12]:
proc=0.75
alla_datum = list(df.datum.unique())
split_ix = int(len(alla_datum)*proc)
y_train=df[split_ix:]['plac']
y_test =df[:split_ix]['plac']
y_train = (y_train==1)*1
y_test  = (y_test==1)*1
X_train = df[split_ix:].copy()
X_test  = df[:split_ix].copy()
X_train.drop(['avd','startnr','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat', 'vodds','podds','dist','bins','streck'   ,'plac'],axis=1,inplace=True)
X_test.drop (['avd','startnr','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat', 'vodds','podds','dist','bins' ,'streck'  ,'plac'],axis=1,inplace=True)
X_train

X_train.shape,X_test.shape,y_train.shape,y_test.shape


((41459, 65), (386, 65), (41459,), (386,))

In [13]:
from sklearn.pipeline import make_pipeline
# FLAML test
from flaml import AutoML 
automl = AutoML( )
# xpipe=make_pipeline(col_pipe, automl(kwargs={'tadk':'classification','time_budget':1000, 'early_stop':True}) )

automl.fit(X_train, y_train,task = 'classification',split_type=None, time_budget=1000, n_jobs=5, early_stop=True, ensemble=True)

[flaml.automl: 09-18 16:32:52] {1427} INFO - Evaluation method: holdout
[flaml.automl: 09-18 16:32:52] {1473} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl: 09-18 16:32:52] {1505} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'lrl1']
[flaml.automl: 09-18 16:32:52] {1735} INFO - iteration 0, current learner lgbm
[flaml.automl: 09-18 16:32:53] {1914} INFO -  at 1.8s,	best lgbm's error=0.3214,	best lgbm's error=0.3214
[flaml.automl: 09-18 16:32:53] {1735} INFO - iteration 1, current learner lgbm
[flaml.automl: 09-18 16:32:53] {1914} INFO -  at 2.4s,	best lgbm's error=0.3214,	best lgbm's error=0.3214
[flaml.automl: 09-18 16:32:53] {1735} INFO - iteration 2, current learner lgbm
[flaml.automl: 09-18 16:32:54] {1914} INFO -  at 3.0s,	best lgbm's error=0.3214,	best lgbm's error=0.3214
[flaml.automl: 09-18 16:32:54] {1735} INFO - iteration 3, current learner xgboost
[flaml.automl: 09-18 16:32:54] {1914} INFO -  at 3.2s,	best xgboost's e

In [14]:
# display(automl.model.get_params())
# display(automl.model)
# display(automl.best_estimator)

train_pred = automl.predict_proba(X_train)
test_pred = automl.predict_proba(X_test)
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.metrics import plot_confusion_matrix, confusion_matrix


In [15]:
print('train auc',roc_auc_score(y_train, train_pred[:,1]))
print('test auc',roc_auc_score(y_test, test_pred[:,1]))
res_train = pd.DataFrame()
res_test  = pd.DataFrame()
res_train['y_train'] = y_train
res_train['pred_train'] = train_pred[:,1]
res_test['y_test'] = y_test
res_test['pred_test'] = test_pred[:,1]

display(f'mean for train=1 {res_train.loc[res_train.y_train==1].pred_train.mean()}')
display(f'mean for train=0 {res_train.loc[res_train.y_train==0].pred_train.mean()}')
display(f'mean for test==1 {res_test.loc[res_test.y_test==1].pred_test.mean()}')
display(f'mean for test==0 {res_test.loc[res_test.y_test==0].pred_test.mean()}')
display()
mätningar = pd.DataFrame()
mätningar['batch'] = [5,4,3,2,1]
mätningar['train_AUC'] = [0.8345, 0.8352,0.8290, 0.8291, 0.8311]
mätningar['test_AUC'] = [0.8418,0.8380,0.8368, 0.8356, 0.8360]
mätningar['avg train win'] =[0.2342,0.2381,0.2267, 0.2241, 0.2258]
mätningar['avg test win'] =[0.2472,0.2377,0.2394, 0.2365, 0.2364]

mätningar


train auc 0.9709107300681581
test auc 0.7182590780324491


'mean for train=1 0.24696779013449016'

'mean for train=0 0.0640084083519145'

'mean for test==1 0.13411174935431194'

'mean for test==0 0.08035785811129295'

,batch,train_AUC,test_AUC,avg train win,avg test win
0,5,0.8345,0.8418,0.2342,0.2472
1,4,0.8352,0.8380,0.2381,0.2377
2,3,0.8290,0.8368,0.2267,0.2394
3,2,0.8291,0.8356,0.2241,0.2365
4,1,0.8311,0.8360,0.2258,0.2364


In [8]:
# automl.model.model.get_feature_importance(prettified=True)
automl.model.model.estimators[0][1].model.get_feature_importance(prettified=True).head(60)

,Feature Id,Importances
0,pris,9.973373
1,datum,8.392651
2,h1_odds,5.986123
3,ekipage,5.966627
4,spår,5.275834
5,kr,5.113219
6,häst,4.580004
7,h1_perf,4.545387
8,h2_odds,3.880331
9,h3_odds,3.087183


## Pickle

In [16]:
import pickle

In [17]:
filename = 'FLAML_model.sav'
pickle.dump(automl, open(filename, 'wb'))

In [20]:
# Open the file in binary mode
with open('FLAML_model.sav', 'rb') as file:
    # Call load method to deserialze
    automl2 = pickle.load(file)
  
    print(automl2)

In [23]:
# from flaml import AutoML 
# automl = AutoML()

alla_datum = list(df.datum.unique())
split_ix = int(len(alla_datum)*0.75)
y_train=df[split_ix:]['plac']
y_test =df[:split_ix]['plac']
y_train = (y_train==1)*1
y_test  = (y_test==1)*1
X_train = df[split_ix:].copy()
X_test  = df[:split_ix].copy()
X_train.drop(['avd','startnr','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat', 'vodds','podds','dist','bins'   ,'plac'],axis=1,inplace=True)
X_test.drop (['avd','startnr','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat', 'vodds','podds','dist','bins'   ,'plac'],axis=1,inplace=True)
# X_train